In [24]:
from Ethan import *
import pandas as pd
import numpy as np

In [25]:
data_frame = pd.read_csv("../data/Titanic/train.csv")
data_frame.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [26]:
from sklearn import preprocessing


def format_feats(in_feats):
    ndArray = in_feats.values
    scaler = preprocessing.MinMaxScaler()
    x_scaled = scaler.fit_transform(ndArray)
    return pd.DataFrame(x_scaled, columns=in_feats.columns)


def format_data(data):
    data = pd.get_dummies(data, columns=["Sex", "Embarked"])
    data = data.drop(labels=["Name", "Ticket", "Cabin", "PassengerId"], axis=1)
    data.fillna(data.mean(), inplace=True)
    if "Survived" in data.columns:
        data_y = data["Survived"]
        data_X = data.drop(labels=["Survived"], axis=1)
        data_X = format_feats(data_X)
        return data_X, data_y
    else:
        return format_feats(data)


feats, labels = format_data(data_frame)
feats.head(4)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.271174,0.125,0.0,0.014151,0.0,1.0,0.0,0.0,1.0
1,0.0,0.472229,0.125,0.0,0.139136,1.0,0.0,1.0,0.0,0.0
2,1.0,0.321438,0.000,0.0,0.015469,1.0,0.0,0.0,0.0,1.0
3,0.0,0.434531,0.125,0.0,0.103644,1.0,0.0,0.0,0.0,1.0


In [27]:
mask = np.random.rand(len(feats)) < 0.8
train_X = feats[mask]
train_y = labels[mask]
test_X = feats[~mask]
test_y = labels[~mask]

train_X = torch.Tensor(train_X.to_numpy()).type(torch.FloatTensor)
train_y = torch.Tensor(train_y.to_numpy()).type(torch.LongTensor)
test_X = torch.Tensor(test_X.to_numpy()).type(torch.FloatTensor)
test_y = torch.Tensor(test_y.to_numpy()).type(torch.LongTensor)

train_dataset = MyDataset(train_X, train_y)
test_dataset = MyDataset(test_X, test_y)

train_dataloader = DataLoader(train_dataset, batch_size=1)
test_dataloader = DataLoader(test_dataset, batch_size=1)

In [ ]:
net = nn.Sequential(nn.Linear(10, 256), nn.ReLU(), nn.Linear(256, 2))
model = MyNetwork(net)

In [28]:
lr = 1e-2
epochs = 10

for t in range(epochs):
    print(f"Epoch {t+1}\n------------------")
    train(train_dataloader, model, lr)
    test(test_dataloader, model)
print("Done!")

# torch.save(model.state_dict(), "../model/Titanic.pth")
# model.load_state_dict(torch.load("../model/Titanic_best.pth"))

Epoch 1
------------------
loss: 1.311670 [    1/  715]
loss: 0.147502 [  144/  715]
loss: 0.342116 [  287/  715]
loss: 0.459781 [  430/  715]
loss: 0.255600 [  573/  715]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.507495 

Epoch 2
------------------
loss: 0.073391 [    1/  715]
loss: 0.051528 [  144/  715]
loss: 0.168948 [  287/  715]
loss: 0.355174 [  430/  715]
loss: 0.169800 [  573/  715]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.496825 

Epoch 3
------------------
loss: 0.062710 [    1/  715]
loss: 0.044278 [  144/  715]
loss: 0.142288 [  287/  715]
loss: 0.322632 [  430/  715]
loss: 0.149594 [  573/  715]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.492730 

Epoch 4
------------------
loss: 0.062506 [    1/  715]
loss: 0.043516 [  144/  715]
loss: 0.135227 [  287/  715]
loss: 0.292334 [  430/  715]
loss: 0.141191 [  573/  715]
Test Error: 
 Accuracy: 79.0%, Avg loss: 0.488905 

Epoch 5
------------------
loss: 0.065461 [    1/  715]
loss: 0.044286 [  144/  715]
loss: 0.133232 [  2

In [29]:
pred_frame = pd.read_csv("../data/Titanic/test.csv")
pred_frame.head(4)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S


In [30]:
feats = format_data(pred_frame)
feats.head(4)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1.0,0.452723,0.000,0.0,0.015282,0.0,1.0,0.0,1.0,0.0
1,1.0,0.617566,0.125,0.0,0.013663,1.0,0.0,0.0,0.0,1.0
2,0.5,0.815377,0.000,0.0,0.018909,0.0,1.0,0.0,1.0,0.0
3,1.0,0.353818,0.000,0.0,0.016908,0.0,1.0,0.0,0.0,1.0


In [31]:
pred_X = torch.Tensor(feats.to_numpy()).type(torch.FloatTensor)

pred_y = pred(pred_X, model)

pred_label = pred_y.argmax(dim=1)
pred_label = pred_label.to("cpu")

In [32]:
pred_Survived = pd.Series(pred_label.numpy())
pred_Survived.name = "Survived"
pred_Survived

0      0
1      0
2      0
3      0
4      0
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [33]:
pred_ID = pred_frame["PassengerId"]
pred_ID

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [34]:
pred = pd.concat([pred_ID, pred_Survived], axis=1)
pred.to_csv("../data/Titanic/pred.csv", index=False)
pred

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
